In [ ]:
"""
Created on Tue Mar 16 10:57 2021

reformat basic mask to give 0-1-2 and floats 
to be used after "data_formatting_smith.sh"

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from cdo import Cdo
import time

In [ ]:
%matplotlib qt5

In [ ]:
cdo = Cdo()
print('this is CDO version %s'%(cdo.version()))

In [ ]:
nemo_run = 'bi646' # 'bf663','bi646'

In [ ]:
inputpath_data='/bettik/burgardc/DATA/NN_PARAM/interim/SMITH_'+nemo_run+'/'
inputpath_raw = '/bettik/burgardc/DATA/NN_PARAM/raw/SMITH_DATA/'

PREPARE THE FLOAT MASK (0 = ocean, 1 = ice shelf, 2 = ground)

In [ ]:
test_TS = xr.open_dataset(inputpath_data + 'TandS_allyy_Ant_withNaN_2041.nc')

In [ ]:
mask_file = xr.open_dataset(inputpath_data+'lsmask_012_timeaxis.nc')
file_map_orig = xr.open_dataset(inputpath_data+'mask_variables_of_interest_allyy_Ant_timeaxis.nc')

In [ ]:
ls_mask012 = mask_file['so'].squeeze().astype('float')

In [ ]:
ls_mask012.to_dataset(name='ls_mask012').to_netcdf(inputpath_data+'custom_lsmask_Ant.nc')

SEPARATE OTHER VARIABLES

In [ ]:
isfdraft = file_map_orig['isf_draft'].where(ls_mask012 < 2, 0).squeeze()#.drop('time')
isfdraft.to_dataset(name='isfdraft').to_netcdf(inputpath_data+'isfdraft_Ant.nc')

In [ ]:
file_map_onlyocean = file_map_orig.where(ls_mask012 < 2, 0)

In [ ]:
check_depth_mask1 = xr.open_dataset(inputpath_raw + 'nemo_bi646c_18801201_mesh_mask.nc')
check_depth_mask2 = xr.open_dataset(inputpath_raw + 'nemo_bi646c_19301201_mesh_mask.nc')

In [ ]:
(check_depth_mask1['gdept_0'] - check_depth_mask2['gdept_0']).max('z').plot() # there are differences below ice shelves but this is no drama for me, because I need the depth for the temperature and salinity in front of the ice shelf

In [ ]:
depth_file = xr.open_dataset(inputpath_data+'3D_depth_coord_Ant.nc')

In [ ]:
other_mask_vars = xr.merge([file_map_onlyocean['Bathymetry_isf'].rename('bathy_metry'), depth_file['gdept_0'].squeeze().drop('time')])
other_mask_vars.to_netcdf(inputpath_data+'other_mask_vars_Ant.nc')

INTERPOLATE TO STEREOGRAPHIC GRID

In [ ]:
print('interpolate float mask')
time_start = time.time()
cdo.remapbil(inputpath_data+'stereo_grid.nc', input = inputpath_data+'custom_lsmask_Ant.nc', output = inputpath_data+'custom_lsmask_Ant_stereo.nc')
timelength = time.time() - time_start
print(timelength)

In [ ]:
print('interpolate isfdraft')
time_start = time.time()
cdo.remapbil(inputpath_data+'stereo_grid.nc', input = inputpath_data+'isfdraft_Ant.nc', output = inputpath_data+'isfdraft_Ant_stereo.nc')
timelength = time.time() - time_start
print(timelength)

PREPARE AN ICE SHELF CONCENTRATION

In [ ]:
isfdraft = xr.open_dataset(inputpath_data+'isfdraft_Ant_stereo.nc')
float_ls_mask = xr.open_dataset(inputpath_data+'custom_lsmask_Ant_stereo.nc')

In [ ]:
# where NaN, set to 2 => land
lsmask_012_withcenter = float_ls_mask['ls_mask012'].where(np.isfinite(float_ls_mask['ls_mask012']),2)
# everything that is not ocean (0) or has a non-zero ice draft, set to 2 => land
should_not_be_ground = (float_ls_mask['ls_mask012']==0) | (isfdraft['isfdraft']>0)
lsmask_below2 = float_ls_mask['ls_mask012'].where(should_not_be_ground,2)

In [ ]:
lsmask_below2.to_dataset(name='ls_mask012').to_netcdf(inputpath_data+'custom_lsmask_Ant_stereo_clean.nc')

In [ ]:
# take only points where there is non-zero ice draft (to get rid of ocean-land transitions)
first_isfdraft_mask = float_ls_mask['ls_mask012'].where(isfdraft['isfdraft']>0,0) 
# if conc is between 0 and 1, that's a transition between ocean and ice so it is an ice-shelf concentration already
# if conc is between 1 and 2, that's transition between ice and land, need to convert it to sth between 0 and 1
isfdraft_conc = first_isfdraft_mask.where(first_isfdraft_mask <= 1,(2 - first_isfdraft_mask))

In [ ]:
isfdraft_conc.isel(time=0).plot()

In [ ]:
isfdraft_conc.to_dataset(name='isfdraft_conc').to_netcdf(inputpath_data+'isfdraft_conc_Ant_stereo.nc')

In [ ]:
# corrected ice draft with ice concentration to come back to the non-smoothed out ice draft
corrected_isfdraft_raw = isfdraft['isfdraft']/isfdraft_conc
# cut unrealistically high drafts
corrected_isfdraft = corrected_isfdraft_raw.where(corrected_isfdraft_raw<=3000,3000).where(corrected_isfdraft_raw>0) 

In [ ]:
corrections = corrected_isfdraft.to_dataset(name='corrected_isfdraft')

REGRIDDING THESE NEW MASK VARIABLES OF INTEREST

In [ ]:
print('computing mask')
time_start = time.time()
cdo.remapbil(inputpath_data+'stereo_grid.nc', input = inputpath_data+'other_mask_vars_Ant.nc', output = inputpath_data+'other_mask_vars_Ant_stereo.nc')
timelength = time.time() - time_start
print(timelength)

In [ ]:
print('creating mean depth coordinate')
time_start = time.time()
cdo.fldmean(input=' -selvar,gdept_0 '+inputpath_data+'other_mask_vars_Ant_stereo.nc', 
          output = inputpath_data+'mask_depth_coord_Ant_stereo.nc') 
timelength = time.time() - time_start
print(timelength)

In [ ]:
mask_vars = xr.open_dataset(inputpath_data+'other_mask_vars_Ant_stereo.nc')

Also correcting the bathymetry but this is not what is used in the study

In [ ]:
mask_vars['bathy_metry'].isel(time=0).plot()

In [ ]:
isfdraft_conc = xr.open_dataset(inputpath_data + 'isfdraft_conc_Ant_stereo.nc')

In [ ]:
corrected_bathy_raw = (mask_vars['bathy_metry']/isfdraft_conc).rename({'isfdraft_conc':'bathy_metry'}) # not realistically cuttable because bathymetry is set to 0 on land...
corrected_bathy = corrected_bathy_raw.where(corrected_bathy_raw<=3000,3000).where(corrected_bathy_raw['bathy_metry']>0)

In [ ]:
corrected_bathy['bathy_metry'].isel(time=0).plot()

In [ ]:
corrected_bathy['bathy_metry'].where(corrected_bathy_raw['bathy_metry']>10000).plot()

In [ ]:
corrections['corrected_isf_bathy'] = corrected_bathy['bathy_metry']

In [ ]:
corrections.to_netcdf(inputpath_data+'corrected_draft_bathy_isf.nc')

In [ ]:
corrections['corrected_isf_bathy'].isel(time=0).plot()

FINISHED? THEN MOVE TO "regridding_vars_cdo_Smith.ipynb"